In [1]:
import time
import datetime
import functions as fs
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

In [4]:
ticker = "^GSPC"
start_date = "2000-01-01"
end_date = "2023-01-01"

day_data = fs.yahoodata(ticker=ticker,
                        period1=start_date,
                        period2=end_date,
                        interval="1d").getdata()
week_data = fs.yahoodata(ticker=ticker,
                        period1=start_date,
                        period2=end_date,
                        interval="1wk").getdata()
month_data = day_data = fs.yahoodata(ticker=ticker,
                        period1=start_date,
                        period2=end_date,
                        interval="1mo").getdata()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
